In [1]:
import os
import time
import tensorflow as tf
from tensorflow.keras import models, layers, Model, callbacks
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from process_data import process_data
from nn_gridsearch import logdir

In [2]:
data = process_data()

In [3]:
for k, v in data.items():
    print(k)
    print(type(v))

x_train
<class 'numpy.ndarray'>
x_train_processed
<class 'numpy.ndarray'>
x_test
<class 'numpy.ndarray'>
x_test_processed
<class 'numpy.ndarray'>
y_train
<class 'numpy.ndarray'>
y_test
<class 'numpy.ndarray'>
pipeline
<class 'sklearn.pipeline.Pipeline'>


In [4]:
x_train_processed = data['x_train_processed']
y_train = data['y_train']

In [5]:
x_train_processed.shape

(1047, 11)

> note: old model constrution, using Sequential API

In [6]:
#model = models.Sequential()
#model.add(layers.Dense(32, activation='relu', input_shape=(x_train_processed.shape[1],)))
#model.add(layers.Dense(32, activation='relu'))
#model.add(layers.Dense(1, activation='sigmoid'))


> changed to functional API --> more flexibility, for future expansion

In [7]:
input_ = layers.Input(shape=x_train_processed.shape[1:])
hidden_0 = layers.Dense(80, activation='relu')(input_)
hidden_1 = layers.Dense(80, activation='relu')(hidden_0)
hidden_2 = layers.Dense(80, activation='relu')(hidden_1)
hidden_3 = layers.Dense(80, activation='relu')(hidden_2)
hidden_4 = layers.Dense(80, activation='relu')(hidden_3)
hidden_5 = layers.Dense(80, activation='relu')(hidden_4)
output = layers.Dense(1, activation='sigmoid')(hidden_5)
model_custom = Model(inputs=[input_], outputs=[output])

In [8]:
#model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model_custom.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', Precision(), Recall(), AUC()])

In [9]:
model_custom.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
dense (Dense)                (None, 80)                960       
_________________________________________________________________
dense_1 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_2 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_3 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_4 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_5 (Dense)              (None, 80)                6480  

In [10]:
history = model_custom.fit(
    data['x_train_processed'], data['y_train'], validation_split=.2, epochs=300,
    callbacks=[
        EarlyStopping(patience=10, monitor='val_loss', mode='min'),
        TensorBoard(logdir('custom_model'))])

Epoch 1/300
27/27 [==============================] - 2s 42ms/step - loss: 0.5990 - accuracy: 0.6595 - precision: 0.5753 - recall: 0.1419 - auc: 0.7003 - val_loss: 0.6181 - val_accuracy: 0.7190 - val_precision: 0.6753 - val_recall: 0.6047 - val_auc: 0.7659
Epoch 2/300
27/27 [==============================] - 1s 20ms/step - loss: 0.4746 - accuracy: 0.8017 - precision: 0.7241 - recall: 0.7095 - auc: 0.8275 - val_loss: 0.5483 - val_accuracy: 0.7429 - val_precision: 0.7162 - val_recall: 0.6163 - val_auc: 0.7987
Epoch 3/300
27/27 [==============================] - 1s 21ms/step - loss: 0.4286 - accuracy: 0.8136 - precision: 0.7917 - recall: 0.6419 - auc: 0.8568 - val_loss: 0.4865 - val_accuracy: 0.7429 - val_precision: 0.7000 - val_recall: 0.6512 - val_auc: 0.8161
Epoch 4/300
27/27 [==============================] - 1s 21ms/step - loss: 0.4189 - accuracy: 0.8256 - precision: 0.8261 - recall: 0.6419 - auc: 0.8601 - val_loss: 0.4951 - val_accuracy: 0.7333 - val_precision: 0.6786 - val_recall: 0

In [11]:
%load_ext tensorboard
%tensorboard --logdir ./custom_logs

In [12]:
model_gridsearch = models.load_model('./models/titanic_gridsearch_best.h5')

In [13]:
model_gridsearch.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden_layer_0_relu_alpha_0. (None, 31)                372       
_________________________________________________________________
batch_normalization_8 (Batch (None, 31)                124       
_________________________________________________________________
activation_8 (Activation)    (None, 31)                0         
_________________________________________________________________
dropout_0_30 (Dropout)       (None, 31)                0         
_________________________________________________________________
hidden_layer_1_relu_alpha_0. (None, 31)                992       
_________________________________________________________________
batch_normalization_9 (Batch (None, 31)                124       
_________________________________________________________________
activation_9 (Activation)    (None, 31)               

In [14]:
model_custom.evaluate(data['x_test_processed'], data['y_test'])

9/9 [==============================] - 0s 17ms/step - loss: 0.4881 - accuracy: 0.7748 - precision: 0.8242 - recall: 0.6356 - auc: 0.8577


[0.4881334900856018,
 0.7748091816902161,
 0.8241758346557617,
 0.6355932354927063,
 0.8576683402061462]

In [19]:
model_gridsearch.evaluate(data['x_test_processed'], data['y_test'])

9/9 [==============================] - 0s 19ms/step - loss: 0.5056 - accuracy: 0.7710 - precision_1: 0.8816 - recall_1: 0.5678 - auc_1: 0.8524


[0.5055856704711914,
 0.7709923386573792,
 0.8815789222717285,
 0.5677965879440308,
 0.8524011373519897]

In [20]:
model_synthetic = models.load_model('./models/titanic_gridsearch_synthetic_2021_10_15_23_42.h5')
model_synthetic.evaluate(data['x_test_processed'], data['y_test'])

9/9 [==============================] - 1s 13ms/step - loss: 0.5070 - accuracy: 0.7786 - precision_1: 0.8846 - recall_1: 0.5847 - auc_1: 0.8276


[0.5069646239280701,
 0.7786259651184082,
 0.8846153616905212,
 0.5847457647323608,
 0.8275953531265259]

In [25]:
model_synthetic_bigger = models.load_model('./models/titanic_gridsearch_synthetic_2021_10_16_05_59.h5')
model_synthetic_bigger.evaluate(data['x_test_processed'], data['y_test'])

9/9 [==============================] - 1s 9ms/step - loss: 0.5190 - accuracy: 0.7672 - precision_1: 0.8701 - recall_1: 0.5678 - auc_1: 0.8065


[0.5189646482467651,
 0.767175555229187,
 0.8701298832893372,
 0.5677965879440308,
 0.8064677715301514]

> todo next - function to build model, sklearn wrapper from keras, pipelines, random grid search for model!

In [24]:
model_synthetic.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden_layer_0_relu_alpha_0. (None, 79)                948       
_________________________________________________________________
activation_8 (Activation)    (None, 79)                0         
_________________________________________________________________
dropout_0_50 (Dropout)       (None, 79)                0         
_________________________________________________________________
hidden_layer_1_relu_alpha_0. (None, 79)                6320      
_________________________________________________________________
activation_9 (Activation)    (None, 79)                0         
_________________________________________________________________
dropout_1_50 (Dropout)       (None, 79)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [26]:
model_synthetic_shrink = models.load_model('./models/titanic_gridsearch_synthetic_2021_10_16_11_17.h5')
model_synthetic_shrink.evaluate(data['x_test_processed'], data['y_test'])

9/9 [==============================] - 1s 14ms/step - loss: 0.5203 - accuracy: 0.7672 - precision_1: 0.8065 - recall_1: 0.6356 - auc_1: 0.8000


[0.5202765464782715,
 0.767175555229187,
 0.8064516186714172,
 0.6355932354927063,
 0.8000234961509705]

In [29]:
model_pure_shrink = models.load_model('models/titanic_gridsearch_2021_10_16_12_26.h5')
model_pure_shrink.evaluate(data['x_test_processed'], data['y_test'])

9/9 [==============================] - 0s 9ms/step - loss: 0.5312 - accuracy: 0.7519 - precision_1: 0.8193 - recall_1: 0.5763 - auc_1: 0.7880


[0.5311660766601562,
 0.7519084215164185,
 0.8192771077156067,
 0.5762711763381958,
 0.7879884243011475]

## experiments - progressively shrink data, compare model performance!

In [34]:
from process_data import process_data
from enhance_data import enhance_data
from nn_gridsearch import nn_gridsearch, make_model
from scipy.stats import reciprocal
from pprint import pprint


#include_synthetic = False
#synthetic_share = 3.0
#real_share = 0.2
    
#data = enhance_data(
#    include_synthetic=include_synthetic,
#    synthetic_share=synthetic_share,
#    real_share=real_share)
    
testing = process_data()
x_test = testing['x_test_processed']
y_test = testing['y_test']
    
grid_parameters = {
    'number_hidden_layers': list(range(1, 8)),
    'neurons': np.arange(1, 100).tolist(),
    'learning_rate': reciprocal(3e-4, 3e-2).rvs(1000).tolist(),
    'dropout_rate': np.arange(.2, .6, .1).tolist(),
    'alpha': np.arange(.2, .35, .05).tolist(),
    'activation': ['elu', 'selu', 'relu']}

models_tested = []

# progressive rate of the full training set
for data_rate in np.linspace(.1, .5, 5):
    
    # test each rate with boosted and non-boosted data
    for boost in (True, False):
        print()
        print('================')
        print(f'creating data: {data_rate} of data boosted: {boost}')
        print(f'real')

        data = enhance_data(real_share=data_rate, synthetic_share=2, include_synthetic=boost)
        
        print(f'real')
        print(f'created training data: {data["x_train_processed"].shape[0]} samples total')
        grid = nn_gridsearch(
            make_model, 
            data['x_train_processed'], data['y_train'],
            grid_parameters,
            n_iterations=33,
            verbose=0)
        
        best_model = grid.best_estimator_.model
        stats = best_model.evaluate(x_test, y_test)
        
        results = {
            'model': best_model,
            'accuracy': stats[1],
            'share_real_data': data_rate,
            'boosted_data': boost,
            'number_training_samples': data['train_y'].shape[0]}
        
        print('finished grid - results:')
        pprint(results)
        models_tested.append(results)
    
    
# note = the wrapper takes a FUNCTION as input!
#grid = nn_gridsearch(
#    make_model, 
#    data['x_train_processed'], data['y_train'],
#    grid_parameters,
#    n_iterations=33,
#    verbose=0)
    
#best_model = grid.best_estimator_.model
#best_model.evaluate(data['x_test_processed'], data['y_test'])


creating data: 0.1 of data boosted: True
Epoch 001 | ET 0.00 min | Avg Losses >> G/D 0.8120/1.2118 [D-Real: 0.5588 D-Fake: 0.6530]
Epoch 002 | ET 0.00 min | Avg Losses >> G/D 0.7807/1.1313 [D-Real: 0.4383 D-Fake: 0.6930]
Epoch 003 | ET 0.01 min | Avg Losses >> G/D 0.8461/1.0301 [D-Real: 0.3768 D-Fake: 0.6533]
Epoch 004 | ET 0.01 min | Avg Losses >> G/D 0.8882/1.0262 [D-Real: 0.3534 D-Fake: 0.6728]
Epoch 005 | ET 0.01 min | Avg Losses >> G/D 0.8737/1.0192 [D-Real: 0.3843 D-Fake: 0.6349]
Epoch 006 | ET 0.01 min | Avg Losses >> G/D 0.8589/0.9506 [D-Real: 0.3046 D-Fake: 0.6460]
Epoch 007 | ET 0.01 min | Avg Losses >> G/D 0.8967/0.9089 [D-Real: 0.3071 D-Fake: 0.6018]
Epoch 008 | ET 0.02 min | Avg Losses >> G/D 0.8223/0.9150 [D-Real: 0.2759 D-Fake: 0.6391]
Epoch 009 | ET 0.02 min | Avg Losses >> G/D 0.8595/0.9463 [D-Real: 0.2810 D-Fake: 0.6653]
Epoch 010 | ET 0.02 min | Avg Losses >> G/D 0.8976/0.8939 [D-Real: 0.2674 D-Fake: 0.6265]
Epoch 011 | ET 0.02 min | Avg Losses >> G/D 0.9680/0.8316 

KeyboardInterrupt: 